In [11]:
import numpy as np, tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

qs = ["hi", "how are you", "what is your name", "bye"]
ans = ["hello", "i am fine", "i am a chatbot", "goodbye"]
ans_in = ["<start> " + a for a in ans]
ans_out = [a + " <end>" for a in ans]

tok = Tokenizer(filters='')
tok.fit_on_texts(qs + ans_in + ans_out)
vocab = len(tok.word_index) + 1
maxlen = max(len(s.split()) for s in qs + ans_in + ans_out)
enc_in = pad_sequences(tok.texts_to_sequences(qs), maxlen=maxlen)
dec_in = pad_sequences(tok.texts_to_sequences(ans_in), maxlen=maxlen)
dec_out = pad_sequences(tok.texts_to_sequences(ans_out), maxlen=maxlen)

ei, di = Input((maxlen,)), Input((maxlen,))
emb = Embedding(vocab, 64)
ee = emb(ei)
de = emb(di)
_, h, c = LSTM(128, return_state=True)(ee)
dl, _, _ = LSTM(128, return_sequences=True, return_state=True)(de, initial_state=[h, c])
out = Dense(vocab, activation='softmax')(dl)
model = Model([ei, di], out)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit([enc_in, dec_in], np.expand_dims(dec_out, -1), epochs=300, verbose=0)

def chat(msg):
    seq = pad_sequences(tok.texts_to_sequences([msg]), maxlen=maxlen)
    dec = np.zeros((1, maxlen)); dec[0, 0] = tok.word_index['<start>']
    for i in range(1, maxlen):
        p = model.predict([seq, dec], verbose=0)
        w = np.argmax(p[0, i-1])
        dec[0, i] = w
        if w == tok.word_index.get('<end>'): break
    txt = tok.sequences_to_texts(dec)[0]
    return txt.replace('<start>', '').replace('<end>', '').strip()

print("User: how are you")
print("Bot :", chat("how are you"))

User: how are you
Bot : i am fine
